In [1]:
import numpy as np
import pandas as pd
import glob, re
import numpy as np
import pandas as pd
from sklearn import *
from datetime import datetime
from xgboost import XGBRegressor


/Users/Jenny/anaconda2/lib/python2.7/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/Users/Jenny/anaconda2/lib/python2.7/site-packages/sklearn/grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)
/Users/Jenny/anaconda2/lib/python2.7/site-packages/sklearn/learning_curve.py:22: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the functions are moved. This module will be removed in 0.20

In [2]:
data = {
    'tra': pd.read_csv('./input/air_visit_data.csv'),
    'as': pd.read_csv('./input/air_store_info.csv'),
    'hs': pd.read_csv('./input/hpg_store_info.csv'),
    'ar': pd.read_csv('./input/air_reserve.csv'),
    'hr': pd.read_csv('./input/hpg_reserve.csv'),
    'id': pd.read_csv('./input/store_id_relation.csv'),
    'tes': pd.read_csv('./input/sample_submission.csv'),
    'hol': pd.read_csv('./input/date_info.csv').rename(columns={'calendar_date':'visit_date'})
    }

In [3]:
data['tra'].head()

air_store_id  visit_date  visitors
0  air_ba937bf13d40fb24  2016-01-13        25
1  air_ba937bf13d40fb24  2016-01-14        32
2  air_ba937bf13d40fb24  2016-01-15        29
3  air_ba937bf13d40fb24  2016-01-16        22
4  air_ba937bf13d40fb24  2016-01-18         6

In [4]:
data['as'].head()

air_store_id  air_genre_name                 air_area_name  \
0  air_0f0cdeee6c9bf3d7  Italian/French  Hyōgo-ken Kōbe-shi Kumoidōri   
1  air_7cc17a324ae5c7dc  Italian/French  Hyōgo-ken Kōbe-shi Kumoidōri   
2  air_fee8dcf4d619598e  Italian/French  Hyōgo-ken Kōbe-shi Kumoidōri   
3  air_a17f0778617c76e2  Italian/French  Hyōgo-ken Kōbe-shi Kumoidōri   
4  air_83db5aff8f50478e  Italian/French  Tōkyō-to Minato-ku Shibakōen   

    latitude   longitude  
0  34.695124  135.197852  
1  34.695124  135.197852  
2  34.695124  135.197852  
3  34.695124  135.197852  
4  35.658068  139.751599

In [5]:
data['hs'].head()

hpg_store_id  hpg_genre_name                  hpg_area_name  \
0  hpg_6622b62385aec8bf  Japanese style  Tōkyō-to Setagaya-ku Taishidō   
1  hpg_e9e068dd49c5fa00  Japanese style  Tōkyō-to Setagaya-ku Taishidō   
2  hpg_2976f7acb4b3a3bc  Japanese style  Tōkyō-to Setagaya-ku Taishidō   
3  hpg_e51a522e098f024c  Japanese style  Tōkyō-to Setagaya-ku Taishidō   
4  hpg_e3d0e1519894f275  Japanese style  Tōkyō-to Setagaya-ku Taishidō   

    latitude   longitude  
0  35.643675  139.668221  
1  35.643675  139.668221  
2  35.643675  139.668221  
3  35.643675  139.668221  
4  35.643675  139.668221

In [6]:
data['ar'].tail()

air_store_id       visit_datetime     reserve_datetime  \
92373  air_db4b38ebe7a7ceff  2017-05-29 18:00:00  2017-04-17 19:00:00   
92374  air_1033310359ceeac1  2017-05-30 23:00:00  2017-04-19 17:00:00   
92375  air_877f79706adbfb06  2017-05-31 20:00:00  2017-04-12 23:00:00   
92376  air_900d755ebd2f7bbd  2017-05-31 20:00:00  2017-04-19 16:00:00   
92377  air_3cad29d1a23209d2  2017-05-31 21:00:00  2017-04-22 10:00:00   

       reserve_visitors  
92373                 6  
92374                 8  
92375                 3  
92376                10  
92377                 3

In [7]:
data['hr'].tail()

hpg_store_id       visit_datetime     reserve_datetime  \
2000315  hpg_2b293477ea4642fb  2017-05-31 20:00:00  2017-04-19 07:00:00   
2000316  hpg_f922aeca48ca5a29  2017-05-31 20:00:00  2017-03-19 00:00:00   
2000317  hpg_e9151de687b93aa3  2017-05-31 21:00:00  2017-04-14 20:00:00   
2000318  hpg_fae6c96189b4a122  2017-05-31 21:00:00  2017-04-19 17:00:00   
2000319  hpg_0b70de808b55ad1e  2017-05-31 23:00:00  2017-04-21 13:00:00   

         reserve_visitors  
2000315                 2  
2000316                 3  
2000317                49  
2000318                 9  
2000319                 6

In [8]:
data['id'].head()

air_store_id          hpg_store_id
0  air_63b13c56b7201bd9  hpg_4bc649e72e2a239a
1  air_a24bf50c3e90d583  hpg_c34b496d0305a809
2  air_c7f78b4f3cba33ff  hpg_cd8ae0d9bbd58ff9
3  air_947eb2cae4f3e8f2  hpg_de24ea49dc25d6b8
4  air_965b2e0cf4119003  hpg_653238a84804d8e7

In [9]:
data['tes'].head(20)

id  visitors
0   air_00a91d42b08b08d9_2017-04-23         0
1   air_00a91d42b08b08d9_2017-04-24         0
2   air_00a91d42b08b08d9_2017-04-25         0
3   air_00a91d42b08b08d9_2017-04-26         0
4   air_00a91d42b08b08d9_2017-04-27         0
5   air_00a91d42b08b08d9_2017-04-28         0
6   air_00a91d42b08b08d9_2017-04-29         0
7   air_00a91d42b08b08d9_2017-04-30         0
8   air_00a91d42b08b08d9_2017-05-01         0
9   air_00a91d42b08b08d9_2017-05-02         0
10  air_00a91d42b08b08d9_2017-05-03         0
11  air_00a91d42b08b08d9_2017-05-04         0
12  air_00a91d42b08b08d9_2017-05-05         0
13  air_00a91d42b08b08d9_2017-05-06         0
14  air_00a91d42b08b08d9_2017-05-07         0
15  air_00a91d42b08b08d9_2017-05-08         0
16  air_00a91d42b08b08d9_2017-05-09         0
17  air_00a91d42b08b08d9_2017-05-10         0
18  air_00a91d42b08b08d9_2017-05-11         0
19  air_00a91d42b08b08d9_2017-05-12         0

In [10]:
data['hol'].head()

visit_date day_of_week  holiday_flg
0  2016-01-01      Friday            1
1  2016-01-02    Saturday            1
2  2016-01-03      Sunday            1
3  2016-01-04      Monday            0
4  2016-01-05     Tuesday            0

In [11]:
#hr表中加上实际店的名称
data['hr'] = pd.merge(data['hr'], data['id'], how='inner', on=['hpg_store_id'])



In [12]:
#对ar、hr两个表做时间处理，增加预定和到店之间的时间差列、对时间差分组增加每个店铺不同日期时间差的和与平均值
for df in ['ar','hr']:
    data[df]['visit_datetime'] = pd.to_datetime(data[df]['visit_datetime'])
    data[df]['visit_datetime'] = data[df]['visit_datetime'].dt.date
    data[df]['reserve_datetime'] = pd.to_datetime(data[df]['reserve_datetime'])
    data[df]['reserve_datetime'] = data[df]['reserve_datetime'].dt.date
    data[df]['reserve_datetime_diff'] = data[df].apply(lambda r: (r['visit_datetime'] - r['reserve_datetime']).days, axis=1)
    tmp1 = data[df].groupby(['air_store_id','visit_datetime'], as_index=False)[['reserve_datetime_diff', 'reserve_visitors']].sum().rename(columns={'visit_datetime':'visit_date', 'reserve_datetime_diff': 'rs1', 'reserve_visitors':'rv1'})
    tmp2 = data[df].groupby(['air_store_id','visit_datetime'], as_index=False)[['reserve_datetime_diff', 'reserve_visitors']].mean().rename(columns={'visit_datetime':'visit_date', 'reserve_datetime_diff': 'rs2', 'reserve_visitors':'rv2'})
    data[df] = pd.merge(tmp1, tmp2, how='inner', on=['air_store_id','visit_date'])


In [13]:
#实际到店信息，添加周几、月、年，更改日期格式
data['tra']['visit_date'] = pd.to_datetime(data['tra']['visit_date'])
data['tra']['dow'] = data['tra']['visit_date'].dt.dayofweek
data['tra']['year'] = data['tra']['visit_date'].dt.year
data['tra']['month'] = data['tra']['visit_date'].dt.month
data['tra']['visit_date'] = data['tra']['visit_date'].dt.date

In [14]:
data['tra'].head()

air_store_id  visit_date  visitors  dow  year  month
0  air_ba937bf13d40fb24  2016-01-13        25    2  2016      1
1  air_ba937bf13d40fb24  2016-01-14        32    3  2016      1
2  air_ba937bf13d40fb24  2016-01-15        29    4  2016      1
3  air_ba937bf13d40fb24  2016-01-16        22    5  2016      1
4  air_ba937bf13d40fb24  2016-01-18         6    0  2016      1

In [15]:
#对测试集第一列变量分裂成店名、日期，并对日期做相应处理
data['tes']['visit_date'] = data['tes']['id'].map(lambda x: str(x).split('_')[2])
data['tes']['air_store_id'] = data['tes']['id'].map(lambda x: '_'.join(x.split('_')[:2]))
data['tes']['visit_date'] = pd.to_datetime(data['tes']['visit_date'])
data['tes']['dow'] = data['tes']['visit_date'].dt.dayofweek
data['tes']['year'] = data['tes']['visit_date'].dt.year
data['tes']['month'] = data['tes']['visit_date'].dt.month
data['tes']['visit_date'] = data['tes']['visit_date'].dt.date

In [16]:
data['tes'].head()

id  visitors  visit_date  \
0  air_00a91d42b08b08d9_2017-04-23         0  2017-04-23   
1  air_00a91d42b08b08d9_2017-04-24         0  2017-04-24   
2  air_00a91d42b08b08d9_2017-04-25         0  2017-04-25   
3  air_00a91d42b08b08d9_2017-04-26         0  2017-04-26   
4  air_00a91d42b08b08d9_2017-04-27         0  2017-04-27   

           air_store_id  dow  year  month  
0  air_00a91d42b08b08d9    6  2017      4  
1  air_00a91d42b08b08d9    0  2017      4  
2  air_00a91d42b08b08d9    1  2017      4  
3  air_00a91d42b08b08d9    2  2017      4  
4  air_00a91d42b08b08d9    3  2017      4

In [17]:
unique_stores = data['tes']['air_store_id'].unique()
stores = pd.concat([pd.DataFrame({'air_store_id': unique_stores, 'dow': [i]*len(unique_stores)}) for i in range(7)], axis=0, ignore_index=True).reset_index(drop=True)
print(len(unique_stores))
stores.tail(20)

821


air_store_id  dow
5727  air_f96765e800907c77    6
5728  air_fa12b40b02fecfd8    6
5729  air_fa4ffc9057812fa2    6
5730  air_fab092c35776a9b1    6
5731  air_fb44f566d4f64a4e    6
5732  air_fbadf737162a5ce3    6
5733  air_fc477473134e9ae5    6
5734  air_fcd4492c83f1c6b9    6
5735  air_fcfbdcf7b1f82c6e    6
5736  air_fd154088b1de6fa7    6
5737  air_fd6aac1043520e83    6
5738  air_fdc02ec4a3d21ea4    6
5739  air_fdcfef8bd859f650    6
5740  air_fe22ef5a9cbef123    6
5741  air_fe58c074ec1445ea    6
5742  air_fea5dc9594450608    6
5743  air_fee8dcf4d619598e    6
5744  air_fef9ccb3ba0da2f7    6
5745  air_ffcc2d5087e1b476    6
5746  air_fff68b929994bfbd    6

In [18]:
#sure it can be compressed...
tmp = data['tra'].groupby(['air_store_id','dow'], as_index=False)['visitors'].min().rename(columns={'visitors':'min_visitors'})
stores = pd.merge(stores, tmp, how='left', on=['air_store_id','dow']) 
tmp = data['tra'].groupby(['air_store_id','dow'], as_index=False)['visitors'].mean().rename(columns={'visitors':'mean_visitors'})
stores = pd.merge(stores, tmp, how='left', on=['air_store_id','dow'])
tmp = data['tra'].groupby(['air_store_id','dow'], as_index=False)['visitors'].median().rename(columns={'visitors':'median_visitors'})
stores = pd.merge(stores, tmp, how='left', on=['air_store_id','dow'])
tmp = data['tra'].groupby(['air_store_id','dow'], as_index=False)['visitors'].max().rename(columns={'visitors':'max_visitors'})
stores = pd.merge(stores, tmp, how='left', on=['air_store_id','dow'])
tmp = data['tra'].groupby(['air_store_id','dow'], as_index=False)['visitors'].count().rename(columns={'visitors':'count_observations'})
stores = pd.merge(stores, tmp, how='left', on=['air_store_id','dow']) 

In [19]:
stores = pd.merge(stores, data['as'], how='left', on=['air_store_id']) 

In [20]:
# NEW FEATURES FROM Georgii Vyshnia
stores['air_genre_name'] = stores['air_genre_name'].map(lambda x: str(str(x).replace('/',' ')))
stores['air_area_name'] = stores['air_area_name'].map(lambda x: str(str(x).replace('-',' ')))

stores.head()

air_store_id  dow  min_visitors  mean_visitors  median_visitors  \
0  air_00a91d42b08b08d9    0           1.0      22.457143             19.0   
1  air_0164b9927d20bcc3    0           2.0       7.500000              6.0   
2  air_0241aa3964b7f861    0           2.0       8.920635              8.0   
3  air_0328696196e46f18    0           2.0       6.416667              4.0   
4  air_034a3d5b40d5b1b1    0           1.0      11.864865             10.0   

   max_visitors  count_observations  air_genre_name  \
0          47.0                35.0  Italian French   
1          19.0                20.0  Italian French   
2          23.0                63.0         Izakaya   
3          27.0                12.0      Dining bar   
4          66.0                37.0     Cafe Sweets   

                     air_area_name   latitude   longitude  
0  Tōkyō to Chiyoda ku Kudanminami  35.694003  139.753595  
1     Tōkyō to Minato ku Shibakōen  35.658068  139.751599  
2    Tōkyō to Taitō ku Higashiueno  35.712607  139.779996  
3     Ōsaka fu Ōsaka shi Nakanochō  34.701279  135.528090  
4       Ōsaka fu Ōsaka shi Ōhiraki  34.692337  135.472229

In [21]:
lbl = preprocessing.LabelEncoder()#标签标准化
for i in range(10):
    stores['air_genre_name'+str(i)] = lbl.fit_transform(stores['air_genre_name'].map(lambda x: str(str(x).split(' ')[i]) if len(str(x).split(' '))>i else ''))
    stores['air_area_name'+str(i)] = lbl.fit_transform(stores['air_area_name'].map(lambda x: str(str(x).split(' ')[i]) if len(str(x).split(' '))>i else ''))

stores.head()

air_store_id  dow  min_visitors  mean_visitors  median_visitors  \
0  air_00a91d42b08b08d9    0           1.0      22.457143             19.0   
1  air_0164b9927d20bcc3    0           2.0       7.500000              6.0   
2  air_0241aa3964b7f861    0           2.0       8.920635              8.0   
3  air_0328696196e46f18    0           2.0       6.416667              4.0   
4  air_034a3d5b40d5b1b1    0           1.0      11.864865             10.0   

   max_visitors  count_observations  air_genre_name  \
0          47.0                35.0  Italian French   
1          19.0                20.0  Italian French   
2          23.0                63.0         Izakaya   
3          27.0                12.0      Dining bar   
4          66.0                37.0     Cafe Sweets   

                     air_area_name   latitude       ...        \
0  Tōkyō to Chiyoda ku Kudanminami  35.694003       ...         
1     Tōkyō to Minato ku Shibakōen  35.658068       ...         
2    Tōkyō to Taitō ku Higashiueno  35.712607       ...         
3     Ōsaka fu Ōsaka shi Nakanochō  34.701279       ...         
4       Ōsaka fu Ōsaka shi Ōhiraki  34.692337       ...         

   air_genre_name5  air_area_name5  air_genre_name6  air_area_name6  \
0                0               0                0               0   
1                0               0                0               0   
2                0               0                0               0   
3                0               0                0               0   
4                0               0                0               0   

   air_genre_name7  air_area_name7  air_genre_name8  air_area_name8  \
0                0               0                0               0   
1                0               0                0               0   
2                0               0                0               0   
3                0               0                0               0   
4                0               0                0               0   

   air_genre_name9  air_area_name9  
0                0               0  
1                0               0  
2                0               0  
3                0               0  
4                0               0  

[5 rows x 31 columns]

In [22]:
stores[['air_genre_name0','air_genre_name1','air_genre_name2','air_genre_name3','air_genre_name4','air_genre_name5']].head()

air_genre_name0  air_genre_name1  air_genre_name2  air_genre_name3  \
0                6                2                0                0   
1                6                2                0                0   
2                7                0                0                0   
3                4                7                0                0   
4                2                6                0                0   

   air_genre_name4  air_genre_name5  
0                0                0  
1                0                0  
2                0                0  
3                0                0  
4                0                0

In [23]:
stores[['air_area_name0','air_area_name1','air_area_name2','air_area_name3','air_area_name4','air_area_name5']].head()

air_area_name0  air_area_name1  air_area_name2  air_area_name3  \
0               7               6               3               6   
1               7               6              26               6   
2               7               6              45               6   
3               8               4              51               7   
4               8               4              51               7   

   air_area_name4  air_area_name5  
0              48               0  
1              78               0  
2              24               0  
3              67               0  
4              97               0

In [24]:
len(stores['air_area_name'].unique())

103

In [25]:
stores['air_genre_name'] = lbl.fit_transform(stores['air_genre_name'])
stores['air_area_name'] = lbl.fit_transform(stores['air_area_name'])
stores.head()

air_store_id  dow  min_visitors  mean_visitors  median_visitors  \
0  air_00a91d42b08b08d9    0           1.0      22.457143             19.0   
1  air_0164b9927d20bcc3    0           2.0       7.500000              6.0   
2  air_0241aa3964b7f861    0           2.0       8.920635              8.0   
3  air_0328696196e46f18    0           2.0       6.416667              4.0   
4  air_034a3d5b40d5b1b1    0           1.0      11.864865             10.0   

   max_visitors  count_observations  air_genre_name  air_area_name   latitude  \
0          47.0                35.0               6             44  35.694003   
1          19.0                20.0               6             62  35.658068   
2          23.0                63.0               7             82  35.712607   
3          27.0                12.0               4             98  34.701279   
4          66.0                37.0               2            102  34.692337   

        ...        air_genre_name5  air_area_name5  air_genre_name6  \
0       ...                      0               0                0   
1       ...                      0               0                0   
2       ...                      0               0                0   
3       ...                      0               0                0   
4       ...                      0               0                0   

   air_area_name6  air_genre_name7  air_area_name7  air_genre_name8  \
0               0                0               0                0   
1               0                0               0                0   
2               0                0               0                0   
3               0                0               0                0   
4               0                0               0                0   

   air_area_name8  air_genre_name9  air_area_name9  
0               0                0               0  
1               0                0               0  
2               0                0               0  
3               0                0               0  
4               0                0               0  

[5 rows x 31 columns]

In [26]:
data['hol']['visit_date'] = pd.to_datetime(data['hol']['visit_date'])
data['hol']['day_of_week'] = lbl.fit_transform(data['hol']['day_of_week'])
data['hol']['visit_date'] = data['hol']['visit_date'].dt.date

In [27]:
data['hol']['day_of_week'].unique()

array([0, 2, 3, 1, 5, 6, 4])

In [28]:
data['hol'].head()

visit_date  day_of_week  holiday_flg
0  2016-01-01            0            1
1  2016-01-02            2            1
2  2016-01-03            3            1
3  2016-01-04            1            0
4  2016-01-05            5            0

In [29]:
train = pd.merge(data['tra'], data['hol'], how='left', on=['visit_date']) 
test = pd.merge(data['tes'], data['hol'], how='left', on=['visit_date']) 

In [30]:
train = pd.merge(train, stores, how='left', on=['air_store_id','dow']) 
test = pd.merge(test, stores, how='left', on=['air_store_id','dow'])

In [31]:
train.head()

air_store_id  visit_date  visitors  dow  year  month  day_of_week  \
0  air_ba937bf13d40fb24  2016-01-13        25    2  2016      1            6   
1  air_ba937bf13d40fb24  2016-01-14        32    3  2016      1            4   
2  air_ba937bf13d40fb24  2016-01-15        29    4  2016      1            0   
3  air_ba937bf13d40fb24  2016-01-16        22    5  2016      1            2   
4  air_ba937bf13d40fb24  2016-01-18         6    0  2016      1            1   

   holiday_flg  min_visitors  mean_visitors       ...        air_genre_name5  \
0            0           7.0      23.843750       ...                    0.0   
1            0           2.0      20.292308       ...                    0.0   
2            0           4.0      34.738462       ...                    0.0   
3            0           6.0      27.651515       ...                    0.0   
4            0           2.0      13.754386       ...                    0.0   

   air_area_name5  air_genre_name6  air_area_name6  air_genre_name7  \
0             0.0              0.0             0.0              0.0   
1             0.0              0.0             0.0              0.0   
2             0.0              0.0             0.0              0.0   
3             0.0              0.0             0.0              0.0   
4             0.0              0.0             0.0              0.0   

   air_area_name7  air_genre_name8  air_area_name8  air_genre_name9  \
0             0.0              0.0             0.0              0.0   
1             0.0              0.0             0.0              0.0   
2             0.0              0.0             0.0              0.0   
3             0.0              0.0             0.0              0.0   
4             0.0              0.0             0.0              0.0   

   air_area_name9  
0             0.0  
1             0.0  
2             0.0  
3             0.0  
4             0.0  

[5 rows x 37 columns]

In [32]:
test.head()

id  visitors  visit_date  \
0  air_00a91d42b08b08d9_2017-04-23         0  2017-04-23   
1  air_00a91d42b08b08d9_2017-04-24         0  2017-04-24   
2  air_00a91d42b08b08d9_2017-04-25         0  2017-04-25   
3  air_00a91d42b08b08d9_2017-04-26         0  2017-04-26   
4  air_00a91d42b08b08d9_2017-04-27         0  2017-04-27   

           air_store_id  dow  year  month  day_of_week  holiday_flg  \
0  air_00a91d42b08b08d9    6  2017      4            3            0   
1  air_00a91d42b08b08d9    0  2017      4            1            0   
2  air_00a91d42b08b08d9    1  2017      4            5            0   
3  air_00a91d42b08b08d9    2  2017      4            6            0   
4  air_00a91d42b08b08d9    3  2017      4            4            0   

   min_visitors       ...        air_genre_name5  air_area_name5  \
0           2.0       ...                      0               0   
1           1.0       ...                      0               0   
2           1.0       ...                      0               0   
3          15.0       ...                      0               0   
4          15.0       ...                      0               0   

   air_genre_name6  air_area_name6  air_genre_name7  air_area_name7  \
0                0               0                0               0   
1                0               0                0               0   
2                0               0                0               0   
3                0               0                0               0   
4                0               0                0               0   

   air_genre_name8  air_area_name8  air_genre_name9  air_area_name9  
0                0               0                0               0  
1                0               0                0               0  
2                0               0                0               0  
3                0               0                0               0  
4                0               0                0               0  

[5 rows x 38 columns]

In [33]:
for df in ['ar','hr']:
    train = pd.merge(train, data[df], how='left', on=['air_store_id','visit_date']) 
    test = pd.merge(test, data[df], how='left', on=['air_store_id','visit_date'])



In [34]:
train.head()


air_store_id  visit_date  visitors  dow  year  month  day_of_week  \
0  air_ba937bf13d40fb24  2016-01-13        25    2  2016      1            6   
1  air_ba937bf13d40fb24  2016-01-14        32    3  2016      1            4   
2  air_ba937bf13d40fb24  2016-01-15        29    4  2016      1            0   
3  air_ba937bf13d40fb24  2016-01-16        22    5  2016      1            2   
4  air_ba937bf13d40fb24  2016-01-18         6    0  2016      1            1   

   holiday_flg  min_visitors  mean_visitors  ...    air_genre_name9  \
0            0           7.0      23.843750  ...                0.0   
1            0           2.0      20.292308  ...                0.0   
2            0           4.0      34.738462  ...                0.0   
3            0           6.0      27.651515  ...                0.0   
4            0           2.0      13.754386  ...                0.0   

   air_area_name9  rs1_x  rv1_x  rs2_x  rv2_x  rs1_y  rv1_y  rs2_y  rv2_y  
0             0.0    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN  
1             0.0    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN  
2             0.0    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN  
3             0.0    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN  
4             0.0    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN  

[5 rows x 45 columns]

In [35]:
test.head()

id  visitors  visit_date  \
0  air_00a91d42b08b08d9_2017-04-23         0  2017-04-23   
1  air_00a91d42b08b08d9_2017-04-24         0  2017-04-24   
2  air_00a91d42b08b08d9_2017-04-25         0  2017-04-25   
3  air_00a91d42b08b08d9_2017-04-26         0  2017-04-26   
4  air_00a91d42b08b08d9_2017-04-27         0  2017-04-27   

           air_store_id  dow  year  month  day_of_week  holiday_flg  \
0  air_00a91d42b08b08d9    6  2017      4            3            0   
1  air_00a91d42b08b08d9    0  2017      4            1            0   
2  air_00a91d42b08b08d9    1  2017      4            5            0   
3  air_00a91d42b08b08d9    2  2017      4            6            0   
4  air_00a91d42b08b08d9    3  2017      4            4            0   

   min_visitors  ...    air_genre_name9  air_area_name9  rs1_x  rv1_x  rs2_x  \
0           2.0  ...                  0               0    NaN    NaN    NaN   
1           1.0  ...                  0               0    NaN    NaN    NaN   
2           1.0  ...                  0               0    NaN    NaN    NaN   
3          15.0  ...                  0               0    NaN    NaN    NaN   
4          15.0  ...                  0               0    NaN    NaN    NaN   

   rv2_x  rs1_y  rv1_y  rs2_y  rv2_y  
0    NaN    NaN    NaN    NaN    NaN  
1    NaN    NaN    NaN    NaN    NaN  
2    NaN    NaN    NaN    NaN    NaN  
3    NaN    NaN    NaN    NaN    NaN  
4    NaN    NaN    NaN    NaN    NaN  

[5 rows x 46 columns]

In [36]:
train['id'] = train.apply(lambda r: '_'.join([str(r['air_store_id']), str(r['visit_date'])]), axis=1)

train['total_reserv_sum'] = train['rv1_x'] + train['rv1_y']
train['total_reserv_mean'] = (train['rv2_x'] + train['rv2_y']) / 2
train['total_reserv_dt_diff_mean'] = (train['rs2_x'] + train['rs2_y']) / 2

test['total_reserv_sum'] = test['rv1_x'] + test['rv1_y']
test['total_reserv_mean'] = (test['rv2_x'] + test['rv2_y']) / 2
test['total_reserv_dt_diff_mean'] = (test['rs2_x'] + test['rs2_y']) / 2



In [37]:
train.head()

air_store_id  visit_date  visitors  dow  year  month  day_of_week  \
0  air_ba937bf13d40fb24  2016-01-13        25    2  2016      1            6   
1  air_ba937bf13d40fb24  2016-01-14        32    3  2016      1            4   
2  air_ba937bf13d40fb24  2016-01-15        29    4  2016      1            0   
3  air_ba937bf13d40fb24  2016-01-16        22    5  2016      1            2   
4  air_ba937bf13d40fb24  2016-01-18         6    0  2016      1            1   

   holiday_flg  min_visitors  mean_visitors            ...              rs2_x  \
0            0           7.0      23.843750            ...                NaN   
1            0           2.0      20.292308            ...                NaN   
2            0           4.0      34.738462            ...                NaN   
3            0           6.0      27.651515            ...                NaN   
4            0           2.0      13.754386            ...                NaN   

   rv2_x  rs1_y  rv1_y  rs2_y  rv2_y                               id  \
0    NaN    NaN    NaN    NaN    NaN  air_ba937bf13d40fb24_2016-01-13   
1    NaN    NaN    NaN    NaN    NaN  air_ba937bf13d40fb24_2016-01-14   
2    NaN    NaN    NaN    NaN    NaN  air_ba937bf13d40fb24_2016-01-15   
3    NaN    NaN    NaN    NaN    NaN  air_ba937bf13d40fb24_2016-01-16   
4    NaN    NaN    NaN    NaN    NaN  air_ba937bf13d40fb24_2016-01-18   

   total_reserv_sum  total_reserv_mean  total_reserv_dt_diff_mean  
0               NaN                NaN                        NaN  
1               NaN                NaN                        NaN  
2               NaN                NaN                        NaN  
3               NaN                NaN                        NaN  
4               NaN                NaN                        NaN  

[5 rows x 49 columns]

In [38]:
test.head()


id  visitors  visit_date  \
0  air_00a91d42b08b08d9_2017-04-23         0  2017-04-23   
1  air_00a91d42b08b08d9_2017-04-24         0  2017-04-24   
2  air_00a91d42b08b08d9_2017-04-25         0  2017-04-25   
3  air_00a91d42b08b08d9_2017-04-26         0  2017-04-26   
4  air_00a91d42b08b08d9_2017-04-27         0  2017-04-27   

           air_store_id  dow  year  month  day_of_week  holiday_flg  \
0  air_00a91d42b08b08d9    6  2017      4            3            0   
1  air_00a91d42b08b08d9    0  2017      4            1            0   
2  air_00a91d42b08b08d9    1  2017      4            5            0   
3  air_00a91d42b08b08d9    2  2017      4            6            0   
4  air_00a91d42b08b08d9    3  2017      4            4            0   

   min_visitors            ...              rv1_x  rs2_x  rv2_x  rs1_y  rv1_y  \
0           2.0            ...                NaN    NaN    NaN    NaN    NaN   
1           1.0            ...                NaN    NaN    NaN    NaN    NaN   
2           1.0            ...                NaN    NaN    NaN    NaN    NaN   
3          15.0            ...                NaN    NaN    NaN    NaN    NaN   
4          15.0            ...                NaN    NaN    NaN    NaN    NaN   

   rs2_y  rv2_y  total_reserv_sum  total_reserv_mean  \
0    NaN    NaN               NaN                NaN   
1    NaN    NaN               NaN                NaN   
2    NaN    NaN               NaN                NaN   
3    NaN    NaN               NaN                NaN   
4    NaN    NaN               NaN                NaN   

   total_reserv_dt_diff_mean  
0                        NaN  
1                        NaN  
2                        NaN  
3                        NaN  
4                        NaN  

[5 rows x 49 columns]

In [122]:
# NEW FEATURES FROM JMBULL
train['date_int'] = train['visit_date'].apply(lambda x: x.strftime('%Y%m%d')).astype(int)
test['date_int'] = test['visit_date'].apply(lambda x: x.strftime('%Y%m%d')).astype(int)

In [123]:
train.head()

air_store_id  visit_date  visitors  dow  year  month  day_of_week  \
0  air_ba937bf13d40fb24  2016-01-13        25    2  2016      1            6   
1  air_ba937bf13d40fb24  2016-01-14        32    3  2016      1            4   
2  air_ba937bf13d40fb24  2016-01-15        29    4  2016      1            0   
3  air_ba937bf13d40fb24  2016-01-16        22    5  2016      1            2   
4  air_ba937bf13d40fb24  2016-01-18         6    0  2016      1            1   

   holiday_flg  min_visitors  mean_visitors    ...     rv2_x  rs1_y  rv1_y  \
0            0           7.0      23.843750    ...       NaN    NaN    NaN   
1            0           2.0      20.292308    ...       NaN    NaN    NaN   
2            0           4.0      34.738462    ...       NaN    NaN    NaN   
3            0           6.0      27.651515    ...       NaN    NaN    NaN   
4            0           2.0      13.754386    ...       NaN    NaN    NaN   

   rs2_y  rv2_y                               id  total_reserv_sum  \
0    NaN    NaN  air_ba937bf13d40fb24_2016-01-13               NaN   
1    NaN    NaN  air_ba937bf13d40fb24_2016-01-14               NaN   
2    NaN    NaN  air_ba937bf13d40fb24_2016-01-15               NaN   
3    NaN    NaN  air_ba937bf13d40fb24_2016-01-16               NaN   
4    NaN    NaN  air_ba937bf13d40fb24_2016-01-18               NaN   

   total_reserv_mean  total_reserv_dt_diff_mean  date_int  
0                NaN                        NaN  20160113  
1                NaN                        NaN  20160114  
2                NaN                        NaN  20160115  
3                NaN                        NaN  20160116  
4                NaN                        NaN  20160118  

[5 rows x 50 columns]

In [124]:
train['var_max_lat'] = train['latitude'].max() - train['latitude']
train['var_max_long'] = train['longitude'].max() - train['longitude']

In [125]:
train.head()

air_store_id  visit_date  visitors  dow  year  month  day_of_week  \
0  air_ba937bf13d40fb24  2016-01-13        25    2  2016      1            6   
1  air_ba937bf13d40fb24  2016-01-14        32    3  2016      1            4   
2  air_ba937bf13d40fb24  2016-01-15        29    4  2016      1            0   
3  air_ba937bf13d40fb24  2016-01-16        22    5  2016      1            2   
4  air_ba937bf13d40fb24  2016-01-18         6    0  2016      1            1   

   holiday_flg  min_visitors  mean_visitors      ...       rv1_y  rs2_y  \
0            0           7.0      23.843750      ...         NaN    NaN   
1            0           2.0      20.292308      ...         NaN    NaN   
2            0           4.0      34.738462      ...         NaN    NaN   
3            0           6.0      27.651515      ...         NaN    NaN   
4            0           2.0      13.754386      ...         NaN    NaN   

   rv2_y                               id  total_reserv_sum  \
0    NaN  air_ba937bf13d40fb24_2016-01-13               NaN   
1    NaN  air_ba937bf13d40fb24_2016-01-14               NaN   
2    NaN  air_ba937bf13d40fb24_2016-01-15               NaN   
3    NaN  air_ba937bf13d40fb24_2016-01-16               NaN   
4    NaN  air_ba937bf13d40fb24_2016-01-18               NaN   

   total_reserv_mean  total_reserv_dt_diff_mean  date_int  var_max_lat  \
0                NaN                        NaN  20160113     8.362564   
1                NaN                        NaN  20160114     8.362564   
2                NaN                        NaN  20160115     8.362564   
3                NaN                        NaN  20160116     8.362564   
4                NaN                        NaN  20160118     8.362564   

   var_max_long  
0      4.521799  
1      4.521799  
2      4.521799  
3      4.521799  
4      4.521799  

[5 rows x 52 columns]

In [126]:
test['var_max_lat'] = test['latitude'].max() - test['latitude']
test['var_max_long'] = test['longitude'].max() - test['longitude']

In [127]:
# NEW FEATURES FROM Georgii Vyshnia
train['lon_plus_lat'] = train['longitude'] + train['latitude'] 
test['lon_plus_lat'] = test['longitude'] + test['latitude']

In [129]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 252108 entries, 0 to 252107
Data columns (total 53 columns):
air_store_id                 252108 non-null object
visit_date                   252108 non-null object
visitors                     252108 non-null int64
dow                          252108 non-null int64
year                         252108 non-null int64
month                        252108 non-null int64
day_of_week                  252108 non-null int64
holiday_flg                  252108 non-null int64
min_visitors                 250468 non-null float64
mean_visitors                250468 non-null float64
median_visitors              250468 non-null float64
max_visitors                 250468 non-null float64
count_observations           250468 non-null float64
air_genre_name               250468 non-null float64
air_area_name                250468 non-null float64
latitude                     250468 non-null float64
longitude                    250468 non-null float64
ai

In [131]:
lbl = preprocessing.LabelEncoder()
train['air_store_id2'] = lbl.fit_transform(train['air_store_id'])
test['air_store_id2'] = lbl.transform(test['air_store_id'])

In [132]:
col = [c for c in train if c not in ['id', 'air_store_id', 'visit_date','visitors']]
col

['dow',
 'year',
 'month',
 'day_of_week',
 'holiday_flg',
 'min_visitors',
 'mean_visitors',
 'median_visitors',
 'max_visitors',
 'count_observations',
 'air_genre_name',
 'air_area_name',
 'latitude',
 'longitude',
 'air_genre_name0',
 'air_area_name0',
 'air_genre_name1',
 'air_area_name1',
 'air_genre_name2',
 'air_area_name2',
 'air_genre_name3',
 'air_area_name3',
 'air_genre_name4',
 'air_area_name4',
 'air_genre_name5',
 'air_area_name5',
 'air_genre_name6',
 'air_area_name6',
 'air_genre_name7',
 'air_area_name7',
 'air_genre_name8',
 'air_area_name8',
 'air_genre_name9',
 'air_area_name9',
 'rs1_x',
 'rv1_x',
 'rs2_x',
 'rv2_x',
 'rs1_y',
 'rv1_y',
 'rs2_y',
 'rv2_y',
 'total_reserv_sum',
 'total_reserv_mean',
 'total_reserv_dt_diff_mean',
 'date_int',
 'var_max_lat',
 'var_max_long',
 'lon_plus_lat',
 'air_store_id2']

In [134]:
train = train.fillna(-1)
test = test.fillna(-1)

In [135]:
def RMSLE(y, pred):
    return metrics.mean_squared_error(y, pred)**0.5

In [136]:
model1 = ensemble.GradientBoostingRegressor(learning_rate=0.2, random_state=3, n_estimators=200, subsample=0.8, 
                      max_depth =10)
model2 = neighbors.KNeighborsRegressor(n_jobs=-1, n_neighbors=4)
model3 = XGBRegressor(learning_rate=0.2, n_estimators=200, subsample=0.8, 
                      colsample_bytree=0.8, max_depth =10)

In [137]:
model1.fit(train[col], np.log1p(train['visitors'].values))
model2.fit(train[col], np.log1p(train['visitors'].values))
model3.fit(train[col], np.log1p(train['visitors'].values))

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.8, gamma=0, learning_rate=0.2, max_delta_step=0,
       max_depth=10, min_child_weight=1, missing=None, n_estimators=200,
       n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=0.8)

In [138]:
preds1 = model1.predict(train[col])
preds2 = model2.predict(train[col])
preds3 = model3.predict(train[col])

In [25]:
import joblib
joblib.dump(model1,'./model1')
joblib.dump(model2,'./model2')
joblib.dump(model3,'./model3')

model1 = joblib.load('./model1')
model2 = joblib.load('./model2')
model3 = joblib.load('./model3')
preds1 = model1.predict(train[col])
preds2 = model2.predict(train[col])
preds3 = model3.predict(train[col])

In [139]:
print('RMSE GradientBoostingRegressor: ', RMSLE(np.log1p(train['visitors'].values), preds1))
print('RMSE KNeighborsRegressor: ', RMSLE(np.log1p(train['visitors'].values), preds2))
print('RMSE XGBRegressor: ', RMSLE(np.log1p(train['visitors'].values), preds3))

('RMSE GradientBoostingRegressor: ', 0.34793105169484123)
('RMSE KNeighborsRegressor: ', 0.41945171611410526)
('RMSE XGBRegressor: ', 0.39827072559893012)


In [140]:
preds1 = model1.predict(test[col])
preds2 = model2.predict(test[col])
preds3 = model3.predict(test[col])

In [141]:
test['visitors'] = 0.3*preds1+0.3*preds2+0.4*preds3

test['visitors'] = np.expm1(test['visitors']).clip(lower=0.)

sub1 = test[['id','visitors']].copy()
del train; del data;

In [143]:
sub1.head()

id   visitors
0  air_00a91d42b08b08d9_2017-04-23   3.326964
1  air_00a91d42b08b08d9_2017-04-24  22.680993
2  air_00a91d42b08b08d9_2017-04-25  29.426445
3  air_00a91d42b08b08d9_2017-04-26  28.134000
4  air_00a91d42b08b08d9_2017-04-27  30.652946

In [147]:
date_info = pd.read_csv('./input/date_info.csv')
print date_info.head()
air_visit_data = pd.read_csv('./input/air_visit_data.csv')
print air_visit_data.head()
sample_submission = pd.read_csv('./input/sample_submission.csv')
print sample_submission.head()

  calendar_date day_of_week  holiday_flg
0    2016-01-01      Friday            1
1    2016-01-02    Saturday            1
2    2016-01-03      Sunday            1
3    2016-01-04      Monday            0
4    2016-01-05     Tuesday            0
           air_store_id  visit_date  visitors
0  air_ba937bf13d40fb24  2016-01-13        25
1  air_ba937bf13d40fb24  2016-01-14        32
2  air_ba937bf13d40fb24  2016-01-15        29
3  air_ba937bf13d40fb24  2016-01-16        22
4  air_ba937bf13d40fb24  2016-01-18         6
                                id  visitors
0  air_00a91d42b08b08d9_2017-04-23         0
1  air_00a91d42b08b08d9_2017-04-24         0
2  air_00a91d42b08b08d9_2017-04-25         0
3  air_00a91d42b08b08d9_2017-04-26         0
4  air_00a91d42b08b08d9_2017-04-27         0


In [156]:
len(dfs.items())

8

In [225]:
# from hklee
# https://www.kaggle.com/zeemeen/weighted-mean-comparisons-lb-0-497-1st/code
dfs = { re.search('/([^/\.]*)\.csv', fn).group(1):
    pd.read_csv(fn)for fn in glob.glob('./input/*.csv')}

In [226]:
for k, v in dfs.items(): locals()[k] = v
    

In [227]:
wkend_holidays = date_info.apply(
    (lambda x:(x.day_of_week=='Sunday' or x.day_of_week=='Saturday') and x.holiday_flg==1), axis=1)

In [228]:
date_info.head(15)


calendar_date day_of_week  holiday_flg
0     2016-01-01      Friday            1
1     2016-01-02    Saturday            1
2     2016-01-03      Sunday            1
3     2016-01-04      Monday            0
4     2016-01-05     Tuesday            0
5     2016-01-06   Wednesday            0
6     2016-01-07    Thursday            0
7     2016-01-08      Friday            0
8     2016-01-09    Saturday            0
9     2016-01-10      Sunday            0
10    2016-01-11      Monday            1
11    2016-01-12     Tuesday            0
12    2016-01-13   Wednesday            0
13    2016-01-14    Thursday            0
14    2016-01-15      Friday            0

In [229]:
wkend_holidays.head(15)

0     False
1      True
2      True
3     False
4     False
5     False
6     False
7     False
8     False
9     False
10    False
11    False
12    False
13    False
14    False
dtype: bool

In [230]:
date_info.loc[wkend_holidays, 'holiday_flg'] = 0
date_info['weight'] = ((date_info.index + 1)*1.0 / len(date_info)) ** 5  

In [231]:
date_info.head(20)

calendar_date day_of_week  holiday_flg        weight
0     2016-01-01      Friday            1  2.707368e-14
1     2016-01-02    Saturday            0  8.663577e-13
2     2016-01-03      Sunday            0  6.578904e-12
3     2016-01-04      Monday            0  2.772345e-11
4     2016-01-05     Tuesday            0  8.460525e-11
5     2016-01-06   Wednesday            0  2.105249e-10
6     2016-01-07    Thursday            0  4.550273e-10
7     2016-01-08      Friday            0  8.871503e-10
8     2016-01-09    Saturday            0  1.598674e-09
9     2016-01-10      Sunday            0  2.707368e-09
10    2016-01-11      Monday            1  4.360243e-09
11    2016-01-12     Tuesday            0  6.736798e-09
12    2016-01-13   Wednesday            0  1.005227e-08
13    2016-01-14    Thursday            0  1.456087e-08
14    2016-01-15      Friday            0  2.055908e-08
15    2016-01-16    Saturday            0  2.838881e-08
16    2016-01-17      Sunday            0  3.844075e-08
17    2016-01-18      Monday            0  5.115756e-08
18    2016-01-19     Tuesday            0  6.703711e-08
19    2016-01-20   Wednesday            0  8.663577e-08

In [232]:
visit_data = air_visit_data.merge(date_info, left_on='visit_date', right_on='calendar_date', how='left')
visit_data.drop('calendar_date', axis=1, inplace=True)
visit_data['visitors'] = visit_data.visitors.map(pd.np.log1p)
print visit_data.info()
visit_data.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 252108 entries, 0 to 252107
Data columns (total 6 columns):
air_store_id    252108 non-null object
visit_date      252108 non-null object
visitors        252108 non-null float64
day_of_week     252108 non-null object
holiday_flg     252108 non-null int64
weight          252108 non-null float64
dtypes: float64(2), int64(1), object(3)
memory usage: 13.5+ MB
None


air_store_id  visit_date  visitors day_of_week  holiday_flg  \
0  air_ba937bf13d40fb24  2016-01-13  3.258097   Wednesday            0   
1  air_ba937bf13d40fb24  2016-01-14  3.496508    Thursday            0   
2  air_ba937bf13d40fb24  2016-01-15  3.401197      Friday            0   
3  air_ba937bf13d40fb24  2016-01-16  3.135494    Saturday            0   
4  air_ba937bf13d40fb24  2016-01-18  1.945910      Monday            0   

         weight  
0  1.005227e-08  
1  1.456087e-08  
2  2.055908e-08  
3  2.838881e-08  
4  5.115756e-08

In [233]:
wmean = lambda x:( (x.weight * x.visitors).sum() / x.weight.sum() )
visitors = visit_data.groupby(['air_store_id', 'day_of_week', 'holiday_flg']).apply(wmean).reset_index()
visitors.rename(columns={0:'visitors'}, inplace=True) # cumbersome, should be better ways.

In [234]:
visitors.head()

air_store_id day_of_week  holiday_flg  visitors
0  air_00a91d42b08b08d9      Friday            0  3.583535
1  air_00a91d42b08b08d9      Monday            0  3.203625
2  air_00a91d42b08b08d9      Monday            1  3.091042
3  air_00a91d42b08b08d9    Saturday            0  2.524065
4  air_00a91d42b08b08d9      Sunday            0  1.098612

In [235]:
sample_submission['air_store_id'] = sample_submission.id.map(lambda x: '_'.join(x.split('_')[:-1]))
sample_submission['calendar_date'] = sample_submission.id.map(lambda x: x.split('_')[2])

In [236]:
sample_submission.head()

id  visitors          air_store_id  \
0  air_00a91d42b08b08d9_2017-04-23         0  air_00a91d42b08b08d9   
1  air_00a91d42b08b08d9_2017-04-24         0  air_00a91d42b08b08d9   
2  air_00a91d42b08b08d9_2017-04-25         0  air_00a91d42b08b08d9   
3  air_00a91d42b08b08d9_2017-04-26         0  air_00a91d42b08b08d9   
4  air_00a91d42b08b08d9_2017-04-27         0  air_00a91d42b08b08d9   

  calendar_date  
0    2017-04-23  
1    2017-04-24  
2    2017-04-25  
3    2017-04-26  
4    2017-04-27

In [237]:
sample_submission.drop('visitors', axis=1, inplace=True)
sample_submission = sample_submission.merge(date_info, on='calendar_date', how='left')
sample_submission = sample_submission.merge(visitors, on=[
    'air_store_id', 'day_of_week', 'holiday_flg'], how='left')

In [238]:
missings = sample_submission.visitors.isnull()
missings

0        False
1        False
2        False
3        False
4        False
5        False
6        False
7        False
8        False
9        False
10        True
11        True
12        True
13       False
14       False
15       False
16       False
17       False
18       False
19       False
20       False
21       False
22       False
23       False
24       False
25       False
26       False
27       False
28       False
29       False
         ...  
31989    False
31990     True
31991    False
31992    False
31993    False
31994    False
31995    False
31996    False
31997    False
31998    False
31999    False
32000    False
32001    False
32002    False
32003    False
32004    False
32005    False
32006    False
32007    False
32008    False
32009    False
32010    False
32011    False
32012    False
32013    False
32014    False
32015    False
32016    False
32017    False
32018    False
Name: visitors, Length: 32019, dtype: bool

In [239]:
sample_submission.loc[missings, 'visitors'] = sample_submission[missings].merge(
    visitors[visitors.holiday_flg==0], on=('air_store_id', 'day_of_week'), 
    how='left')['visitors_y'].values

In [240]:
missings = sample_submission.visitors.isnull()

In [241]:
sample_submission.loc[missings, 'visitors'] = sample_submission[missings].merge(
    visitors[['air_store_id', 'visitors']].groupby('air_store_id').mean().reset_index(), 
    on='air_store_id', how='left')['visitors_y'].values

In [242]:
sample_submission.head()

id          air_store_id calendar_date  \
0  air_00a91d42b08b08d9_2017-04-23  air_00a91d42b08b08d9    2017-04-23   
1  air_00a91d42b08b08d9_2017-04-24  air_00a91d42b08b08d9    2017-04-24   
2  air_00a91d42b08b08d9_2017-04-25  air_00a91d42b08b08d9    2017-04-25   
3  air_00a91d42b08b08d9_2017-04-26  air_00a91d42b08b08d9    2017-04-26   
4  air_00a91d42b08b08d9_2017-04-27  air_00a91d42b08b08d9    2017-04-27   

  day_of_week  holiday_flg    weight  visitors  
0      Sunday            0  0.682692  1.098612  
1      Monday            0  0.689848  3.203625  
2     Tuesday            0  0.697064  3.325868  
3   Wednesday            0  0.704340  3.353439  
4    Thursday            0  0.711677  3.475056

In [243]:
np.exp(1.098612)

2.9999991339957957

In [244]:
sample_submission['visitors'] = sample_submission.visitors.map(pd.np.expm1)
sample_submission.head()

id          air_store_id calendar_date  \
0  air_00a91d42b08b08d9_2017-04-23  air_00a91d42b08b08d9    2017-04-23   
1  air_00a91d42b08b08d9_2017-04-24  air_00a91d42b08b08d9    2017-04-24   
2  air_00a91d42b08b08d9_2017-04-25  air_00a91d42b08b08d9    2017-04-25   
3  air_00a91d42b08b08d9_2017-04-26  air_00a91d42b08b08d9    2017-04-26   
4  air_00a91d42b08b08d9_2017-04-27  air_00a91d42b08b08d9    2017-04-27   

  day_of_week  holiday_flg    weight   visitors  
0      Sunday            0  0.682692   2.000000  
1      Monday            0  0.689848  23.621632  
2     Tuesday            0  0.697064  26.823130  
3   Wednesday            0  0.704340  27.600920  
4    Thursday            0  0.711677  31.299646

In [245]:
sub2 = sample_submission[['id', 'visitors']].copy()
sub_merge = pd.merge(sub1, sub2, on='id', how='inner')
sub_merge.head()

id  visitors_x  visitors_y
0  air_00a91d42b08b08d9_2017-04-23    3.326964    2.000000
1  air_00a91d42b08b08d9_2017-04-24   22.680993   23.621632
2  air_00a91d42b08b08d9_2017-04-25   29.426445   26.823130
3  air_00a91d42b08b08d9_2017-04-26   28.134000   27.600920
4  air_00a91d42b08b08d9_2017-04-27   30.652946   31.299646

In [246]:
sub_merge['visitors'] = 0.7*sub_merge['visitors_x'] + 0.3*sub_merge['visitors_y']* 1.1
sub_merge[['id', 'visitors']].to_csv('submission.csv', index=False)